This notebook contains the code for the final project of CS 344 Artificial Intelligence at Calvin College.

It creates a number of different networks, all aimed at using the Stanford Cars dataset to identify car brands.

In [1]:
'''
Created on 22 sty 2018
3D Object Representations for Fine-Grained Categorization
Jonathan Krause, Michael Stark, Jia Deng, Li Fei-Fei
4th IEEE Workshop on 3D Representation and Recognition, at ICCV 2013 (3dRR-13). Sydney, Australia. Dec. 8, 2013.
@author: mgdak
'''

import os
import argparse
import sys
import random
import shutil as sh
from pprint import pprint
import scipy.io as sio
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.models import Model, Sequential, model_from_json
import shutil as sh
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121
from keras import callbacks
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16
from keras.optimizers import SGD, RMSprop
from matplotlib import pyplot as plt
from keras import regularizers
from keras.layers.core import Flatten
from keras.layers.normalization import BatchNormalization
from keras import backend as K

train_data_dir = 'cars_train'
val_data_dir = 'cars_val'
nb_train_samples = 1043
nb_val_samples = 300

Using TensorFlow backend.


Imports and Global Variable Definitions

In [2]:
def evaluate(learning_rate, lr_decay):
    test_datagen = ImageDataGenerator(rescale=1. / 255)
    test_generator = test_datagen.flow_from_directory(
        "/home/cjw44/allCars/car_ims/cars_test",
        target_size=(224, 224),
        batch_size=16,
        class_mode='categorical')

    # load json and create model
    json_file = open("./carRecognition_finalModel" + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("./carRecognition_finalModel" + '.h5')
    print('Loaded model from disk')

    # evaluate loaded model on test data
    sgd = SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
    loaded_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    score = loaded_model.evaluate_generator(test_generator, 3957 / 16, workers=6)
    print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1] * 100))

# Creates an array with following values: picture name, picture category ID, train/validation label       
def readData(matFile):
    content = sio.loadmat(matFile)
    data = [(_[0][0][:], _[5][0][0], _[6][0][0]) for _ in content['annotations'][0]]
    return data


# Creates an array of all classes
def readClasses(matFile):
    content = sio.loadmat(matFile)
    classes = [(_[0]) for _ in content['class_names'][0]]
    return classes


# Movces raw data (pictures) into respective category subfolders with train/validation division 
def dataPreprocessing(dataDir, labelsFile):
    data = readData(labelsFile)
    classes = readClasses(labelsFile)
    print("---------------")
    for recData in data:
        if recData[2] == 1:
            # validation set
            os.makedirs(dataDir + "/" + val_data_dir + "/" + classes[recData[1] - 1] + "/", exist_ok=True)
            sh.move(dataDir + "/" + recData[0][8:],
                    dataDir + "/" + val_data_dir + "/" + classes[recData[1] - 1] + "/" + recData[0][8:])
        else:
            os.makedirs(dataDir + "/" + train_data_dir + "/" + classes[recData[1] - 1] + "/", exist_ok=True)
            sh.move(dataDir + "/" + recData[0][8:],
                    dataDir + "/" + train_data_dir + "/" + classes[recData[1] - 1] + "/" + recData[0][8:])  # train set


# serializes the trained model and its weights
def serializeModel(model, fileName):
    # serialize model to JSON
    model_json = model.to_json()
    with open(fileName + ".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights(fileName + ".h5")
    print("Saved model to disk")


def prepareDataGenerators(batchSize, srcImagesDir, labelsFile):
    classes = readClasses(labelsFile)
    # this is the augmentation configuration used for training
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
    # this is the augmentation configuration used for testing:
    # only rescaling
    test_datagen = ImageDataGenerator(rescale=1. / 255)
    # this is a generator that will read pictures found in
    # subfolers of 'car_ims_dir/train', and indefinitely generate
    # batches of augmented image data
    train_generator = train_datagen.flow_from_directory(
        srcImagesDir + "/" + train_data_dir + "/",  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 299x299
        batch_size=batchSize,
        class_mode='categorical')  # since we use categorical_crossentropy loss, we need categorical labels
    # this is a similar generator, for validation data
    validation_generator = test_datagen.flow_from_directory(
        srcImagesDir + "/" + val_data_dir + "/",
        target_size=(224, 224),
        batch_size=batchSize,
        class_mode='categorical')
    return classes, train_generator, validation_generator

Define functions that allow for training and using neural networks

In [3]:
def getVGG16Architecture(classes, dropoutRate):
    # create the base pre-trained model
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in enumerate(base_model.layers):
        layer[1].trainable = False

    # flatten the results from conv block
    x = Flatten()(base_model.output)

    # add another fully connected layers with batch norm and dropout
    x = Dense(4096, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropoutRate)(x)

    # add another fully connected layers with batch norm and dropout
    x = Dense(4096, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropoutRate)(x)

    # add logistic layer with all car classes
    predictions = Dense(20, activation='softmax', kernel_initializer='random_uniform',
                        bias_initializer='random_uniform', bias_regularizer=regularizers.l2(0.01), name='predictions')(
        x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    return model


def getVGG19Architecture(classes, dropoutRate):
    # create the base pre-trained model
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in enumerate(base_model.layers):
        layer[1].trainable = False

    # flatten the results from conv block
    x = Flatten()(base_model.output)

    # add another fully connected layers with batch norm and dropout
    x = Dense(4096, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropoutRate)(x)

    # add another fully connected layers with batch norm and dropout
    x = Dense(4096, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropoutRate)(x)

    # add logistic layer with all car classes
    predictions = Dense(20, activation='softmax', kernel_initializer='random_uniform',
                        bias_initializer='random_uniform', bias_regularizer=regularizers.l2(0.01), name='predictions')(
        x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    return model


def getInceptionV3Architecture(classes, dropoutRate):
    # create the base pre-trained model
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # InceptionV3
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # let's add a fully-connected layer
    x = Dense(1024, activation='relu', kernel_initializer='random_uniform', bias_initializer='random_uniform',
              bias_regularizer=regularizers.l2(0.01))(x)

    # add Dropout regularizer
    x = Dropout(dropoutRate)(x)

    # and a logistic layer with all car classes
    predictions = Dense(len(classes), activation='softmax', kernel_initializer='random_uniform',
                        bias_initializer='random_uniform', bias_regularizer=regularizers.l2(0.01), name='predictions')(
        x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in enumerate(base_model.layers):
        layer[1].trainable = False

    return model

def getMyCNN(classes, dropoutRate):
    model = Sequential()

    model.add(Conv2D(256, 4, activation='relu', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(256, 4, activation='relu'))
    model.add(Conv2D(512, 4, activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(512, 4, activation='relu'))
    model.add(Conv2D(512, 4, activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())

    model.add(Dense(256, activation='relu', kernel_initializer='random_uniform', bias_initializer='random_uniform',
                    bias_regularizer=regularizers.l2(0.01)))

    model.add(Dense(20, activation='softmax', kernel_initializer='random_uniform',
                        bias_initializer='random_uniform', bias_regularizer=regularizers.l2(0.01), name='predictions'))

    return model


Functions that define the neural networks

In [6]:
def setLayersToRetrain(model, modelArchitecture):
    if modelArchitecture == 'InceptionV3':
        # we chose to train the top 2 inception blocks, i.e. we will freeze
        # the first 249 layers and unfreeze the rest:
        for layer in model.layers[:249]:
            layer.trainable = False

        for layer in model.layers[249:]:
            layer.trainable = True
    elif modelArchitecture == 'VGG16':
        # train the last conv block
        for layer in model.layers[:15]:
            layer.trainable = False

        for layer in model.layers[15:]:
            layer.trainable = True
    elif modelArchitecture == 'VGG19':
        # train the last conv block
        for layer in model.layers[:17]:
            layer.trainable = False

        for layer in model.layers[17:]:
            layer.trainable = True
    else:
        for layer in model.layers[:5]:
            layer.trainable = False
        for layer in model.layers[5:]:
            layer.trainable = True


def initialTraining(optimizerLastLayer, noOfEpochs, batchSize, savedModelName, train_generator, validation_generator,
                    model, modelArchitecture, lr_decay, learningRate):
    # compile the model and train the top layer only

    rms = RMSprop(decay=lr_decay, lr=learningRate)
    model.compile(optimizer=rms, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    earlystop = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, mode='auto')
    history = model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batchSize,
        epochs=noOfEpochs,
        validation_data=validation_generator,
        validation_steps=nb_val_samples // batchSize,
        callbacks=[earlystop])
    plt.plot(history.history['val_acc'], 'r')
    plt.plot(history.history['acc'], 'b')
    plt.title('Performance of model ' + modelArchitecture)
    plt.ylabel('Accuracy')
    plt.xlabel('Epochs No')
    plt.savefig(savedModelName + '_initialModel_plot.png')
    serializeModel(model, savedModelName + "_initialModel")


def finetuningTraining(learningRate, noOfEpochs, batchSize, savedModelName, train_generator, validation_generator,
                       model, lr_decay):
    # we need to recompile the model for these modifications to take effect
    # we use SGD with a low learning rate
    sgd = SGD(lr=learningRate, decay=lr_decay, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    earlystop = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, mode='auto')

    # we train our model again (this time fine-tuning the top 2 inception blocks
    # alongside the top Dense layers
    history = model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batchSize,
        epochs=noOfEpochs,
        validation_data=validation_generator,
        validation_steps=nb_val_samples // batchSize,        callbacks=[earlystop])
    plt.clf()
    plt.plot(history.history['val_acc'], 'r')
    plt.plot(history.history['acc'], 'b')
    plt.savefig(savedModelName + '_finalModel_plot.png')
    serializeModel(model, savedModelName + "_finalModel")


def model(learningRate, optimizerLastLayer, noOfEpochs, batchSize, savedModelName, srcImagesDir, labelsFile,
          modelArchitecture, dropoutRate, lr_decay):
    classes, train_generator, validation_generator = prepareDataGenerators(batchSize, srcImagesDir, labelsFile)

    if modelArchitecture == 'VGG16':
        model = getVGG16Architecture(classes, dropoutRate)
    elif modelArchitecture == 'VGG19':
        model = getVGG19Architecture(classes, dropoutRate)
    elif modelArchitecture == "InceptionV3":
        model = getInceptionV3Architecture(classes, dropoutRate)
    else:
        model = getMyCNN(classes, dropoutRate)

    initialTraining(optimizerLastLayer, noOfEpochs, batchSize, savedModelName, train_generator, validation_generator,
                    model, modelArchitecture, lr_decay, learningRate)

    setLayersToRetrain(model, modelArchitecture)


    finetuningTraining(learningRate, noOfEpochs, batchSize, savedModelName, train_generator, validation_generator,
                       model, lr_decay)

    evaluate(learningRate, lr_decay)

Functions used to run the model

In [13]:
def main(args):
    pprint(args)
    if args["process_data"]:
        dataPreprocessing(args["car_ims_dir"], args["car_ims_labels"])

    K.clear_session()

    model(args["learning_rate"],
          args["optimizer_last_layer"],
          args["no_of_epochs"],
          args["batch_size"],
          args["saved_model_name"],
          args["car_ims_dir"],
          args["car_ims_labels"],
          args["model"],
          args["dropout_rate"],
          args["lr_decay"])

Main function for starting model

In [ ]:
args = {}

args["process_data"] = False
args["car_ims_dir"] = "/home/cjw44/allCars/car_ims"
args["car_ims_labels"] = "/home/cjw44/CarData/cars_annos"
args["learning_rate"] = 0.001
args["dropout_rate"] = 0.7
args["lr_decay"] = 1e-4
args["optimizer_last_layer"] = "RMSPROP"  # options: ['ADAGRAD', 'ADADELTA', 'ADAM', 'RMSPROP', 'MOM']
args["model"] = "MyCNN" # options: ['VGG19', 'VGG16', 'InceptionV3', "MyCNN"]
args["no_of_epochs"] = 3
args["batch_size"] = 64
args["saved_model_name"] = "carRecognition"

main(args)

{'batch_size': 64,
 'car_ims_dir': '/home/cjw44/allCars/car_ims',
 'car_ims_labels': '/home/cjw44/CarData/cars_annos',
 'dropout_rate': 0.7,
 'learning_rate': 0.001,
 'lr_decay': 0.0001,
 'model': 'MyCNN',
 'no_of_epochs': 3,
 'optimizer_last_layer': 'RMSPROP',
 'process_data': False,
 'saved_model_name': 'carRecognition'}
Found 1043 images belonging to 20 classes.
Found 331 images belonging to 20 classes.
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 221, 221, 256)     12544     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 256)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 107, 107, 256)     1048832   
________________________________________________________